In [1]:
pip install -r "requirement.txt"

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sqlite3
import pandas as pd
db_file = "Fund.db"
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 'PYPL', 'ADBE', 'INTC']#tickers contenus dans MAIN_DATA_FRAME

start_date = "01/09/2022"#toute la periode start date (pour tout telecharger dun coup) + 3 mois avant
end_date = "31/12/2024"#pareil

## 1. Creation et peuplement de la base

In [3]:
from base_builder import DatabaseBuilder

In [4]:
db_manager = DatabaseBuilder(db_file)
db_manager.create_tables()
db_manager.insert_clients_data(12)
db_manager.insert_managers_data(3)

Tables created successfully.
12 clients insérés avec succès dans la table 'Clients'.
3 managers insérés avec succès dans la table 'Managers'.


In [5]:
db_manager.get_investment_amount_by_risk_type()
db_manager.insert_initial_cash_portfolios(start_date)


Résumé des investissements par risk type:
--------------------------------------------------
Risk Type: HY_EQUITY
Nombre de clients: 2
Montant total investi: 5,822,049.34 €
--------------------------------------------------
Risk Type: LOW_RISK
Nombre de clients: 4
Montant total investi: 12,195,668.05 €
--------------------------------------------------
Risk Type: LOW_TURNOVER
Nombre de clients: 6
Montant total investi: 31,689,767.94 €
--------------------------------------------------
Montants initiaux en cash insérés avec succès dans la table Portfolios.


## 2. Collecte et traitement des datas

In [6]:
from data_collector import GetData

In [7]:
gdata = GetData(tickers, start_date, end_date)
main_df =gdata.main_data_frame()  

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


# 3. Periode avant t = 0 (Periode de "test")

On peuple Products avec des trois mois de donnees avant la periode d'evaluation du fonds (qui commence le 01/01/2023)

In [8]:
from base_update import BaseUpdate 
sub_start_date = "01/09/2022"#chaque semaine, les strat lancent la fct et definissent les sub périodes
#de cette maniere, products ne contient que la data jusqu'à la date t (no forward looking sur nos strats)
sub_end_date = "29/12/2022"

In [9]:
B_udt = BaseUpdate(tickers, sub_start_date, sub_end_date, db_file, main_df)
B_udt.update_products()

Successfully inserted 830 new records into Products table for the period 01/09/2022 to 29/12/2022


### Initialisation des portefeuilles en fonction du cash initial

Pour la strategie HY equity, on prend les 5 tickers les plus performants de la periode (allant du 01/09/2022 au 30/12/2022)

In [10]:
B_udt.initialisation_portefeuille_HY()

Cash disponible : 5,822,049.34

Tickers sélectionnés pour l'initialisation :
AAPL: 155.89 (Rendement: 0.00%)
MSFT: 254.96 (Rendement: 0.00%)
GOOGL: 109.22 (Rendement: 0.00%)
AMZN: 127.82 (Rendement: 0.00%)
META: 164.58 (Rendement: 0.00%)
→ Achat de 7469 unités de AAPL @ 155.89
→ Achat de 4566 unités de MSFT @ 254.96
→ Achat de 10661 unités de GOOGL @ 109.22
→ Achat de 9109 unités de AMZN @ 127.82
→ Achat de 7074 unités de META @ 164.58

Cash restant : 617.03


# 4. Execution des strategies

In [11]:
from strategies import RunAllStrat
start_date2="01/01/2023"
end_date2="31/12/2023"
b=RunAllStrat(db_file,start_date2,end_date2,tickers, main_df)
b.run()


Début de la simulation historique du 01/01/2023 au 31/12/2023

=== Simulation pour le lundi 02/01/2023 ===
Exécution de la stratégie pour la date historique : 02/01/2023
Updating products for historical week: 26/12/2022 to 01/01/2023
Successfully inserted 40 new records into Products table for the period 26/12/2022 to 01/01/2023
Running strategies...
Running HY_EQUITY strategy...
→ Transaction effectuée : Sell 2654 x AMZN @ 84.00
→ Transaction effectuée : Sell 1145 x AAPL @ 128.44
→ Transaction effectuée : Sell 1084 x META @ 119.78
→ Transaction effectuée : Sell 1186 x GOOGL @ 87.81
→ Transaction effectuée : Buy 29 x MSFT @ 235.48
Cash restant : 597,763.73

=== Simulation pour le lundi 09/01/2023 ===
Exécution de la stratégie pour la date historique : 09/01/2023
Updating products for historical week: 02/01/2023 to 08/01/2023
Successfully inserted 40 new records into Products table for the period 02/01/2023 to 08/01/2023
Running strategies...
Running HY_EQUITY strategy...
→ Transaction

In [ ]:
import sqlite3

db_path = "Fund.db"  # adapte si ton fichier a un chemin différent

create_history_table_query = """
CREATE TABLE IF NOT EXISTS Portfolio_History (
    HISTORY_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    MANAGER_ID INTEGER NOT NULL,
    TICKER TEXT NOT NULL,
    QUANTITY INTEGER NOT NULL,
    DATE_SNAPSHOT DATE NOT NULL,
    FOREIGN KEY(MANAGER_ID) REFERENCES Managers(MANAGER_ID) ON DELETE CASCADE
);
"""

conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute(create_history_table_query)
conn.commit()
conn.close()

print("✅ Table Portfolio_History créée avec succès.")


In [ ]:
def rebuild_portfolio_history_by_manager(self, manager_id):
    try:
        conn = sqlite3.connect(self.db_file)
        cursor = conn.cursor()

        # 1. Composition initiale : actifs dans Portfolios pour ce manager
        cursor.execute("""
            SELECT TICKER, QUANTITY
            FROM Portfolios
            WHERE MANAGER_ID = ?
        """, (manager_id,))
        initial_assets = cursor.fetchall()
        portfolio = {ticker: qty for ticker, qty in initial_assets}

        # 2. Récupérer tous les deals du manager, triés par date
        cursor.execute("""
            SELECT EXECUTION_DATE, TICKER, TRADE_TYPE, QUANTITY
            FROM Deals
            WHERE MANAGER_ID = ?
            ORDER BY EXECUTION_DATE ASC
        """, (manager_id,))
        deals = cursor.fetchall()

        # 3. Appliquer chaque deal et sauvegarder un snapshot
        for deal in deals:
            date, ticker, trade_type, quantity = deal

            if trade_type == 'Buy':
                portfolio[ticker] = portfolio.get(ticker, 0) + quantity
            elif trade_type == 'Sell':
                portfolio[ticker] = max(portfolio.get(ticker, 0) - quantity, 0)

            # Retirer les actifs avec 0
            portfolio = {k: v for k, v in portfolio.items() if v > 0}

            # 4. Insérer un snapshot complet pour cette date
            for tick, qty in portfolio.items():
                cursor.execute("""
                    INSERT INTO Portfolio_History (MANAGER_ID, TICKER, QUANTITY, DATE_SNAPSHOT)
                    VALUES (?, ?, ?, ?)
                """, (manager_id, tick, qty, date))

        conn.commit()
        print(f"📈 Historique reconstruit pour le manager {manager_id}")
    except sqlite3.Error as e:
        print(f"❌ Erreur SQLite : {e}")
    finally:
        if conn:
            conn.close()


In [13]:
db_manager.rebuild_portfolio_history_by_manager(manager_id=1)


📈 Historique reconstruit pour le manager 1
